In [2]:
'''
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config=ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction=0.5
config.gpu_options.allow_growth=True
session=InteractiveSession(config=config)'''

In [1]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
from glob import glob

In [2]:
Image_size=[224,224]
train_path='data/train'
test_path='data/test'

In [3]:
resnet=ResNet50(input_shape=Image_size+[3],weights='imagenet',include_top=False)

In [4]:
for layer in resnet.layers:
    layer.trainable=False


In [5]:
folders=glob('data/train/*')

In [6]:
x=Flatten()(resnet.output)

In [7]:
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=resnet.input,outputs=prediction)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
training_set=train_datagen.flow_from_directory('data/train',
                                               target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 1951 images belonging to 4 classes.


In [12]:
test_set=test_datagen.flow_from_directory('data/test',
                                               target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 18 images belonging to 4 classes.


In [13]:
r=model.fit(
training_set,
validation_data=test_set,
epochs=10,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

Train for 61 steps, validate for 1 steps
Epoch 1/10
61/61 [==============================] - 49s 796ms/step - loss: 2.3224 - accuracy: 0.8355 - val_loss: 4.5395 - val_accuracy: 0.1667
Epoch 2/10
61/61 [==============================] - 40s 653ms/step - loss: 0.8594 - accuracy: 0.9344 - val_loss: 9.3679 - val_accuracy: 0.2778
Epoch 3/10
61/61 [==============================] - 38s 625ms/step - loss: 0.5460 - accuracy: 0.9539 - val_loss: 11.0041 - val_accuracy: 0.1667
Epoch 4/10
61/61 [==============================] - 38s 619ms/step - loss: 0.3382 - accuracy: 0.9733 - val_loss: 11.7662 - val_accuracy: 0.1667
Epoch 5/10
61/61 [==============================] - 37s 609ms/step - loss: 0.3820 - accuracy: 0.9682 - val_loss: 8.5824 - val_accuracy: 0.1667
Epoch 6/10
61/61 [==============================] - 37s 608ms/step - loss: 0.4069 - accuracy: 0.9744 - val_loss: 11.8302 - val_accuracy: 0.1667
Epoch 7/10
61/61 [==============================] - 37s 605ms/step - loss: 0.3620 - accuracy: 0.97

In [14]:
from tensorflow.keras.models import load_model

model.save('model_fit_resnet50.h5')

In [14]:
r_1=model.fit_generator(
training_set,
validation_data=test_set,
epochs=10,
steps_per_epoch=len(training_set)
)

Epoch 1/10


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]